Since `Chang14` light schedules with small `dt` take long to generate, we calculate them once and save the results. This notebook generates the time grids and saves them in `data/light_schedules/`.

In [15]:
import os
import copy as cp
import numpy as np
from circadian.lights import LightSchedule
from concurrent.futures import ThreadPoolExecutor

In [16]:
save_path = "../data/light_schedules"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [17]:
dt = 0.005 # hours
days = 14.0
time = np.arange(0, 24.0 * days, dt)

In [18]:
regular_parameters = {
    "lux": 1000.0,
    "lights_on": 6.0,
    "lights_off": 22.0,
}
regular_save_name = "regular"
chang14_parameters = {
    "dim_lux": 3.0,
    "ereader_lux": 31.73,
    "book_lux": 0.91,
    "reading_start_time": 18.0,
    "reading_duration": 4.0,
}

typical_indoor_lux_options = [0.03, 90, 180, 500, 1260, 9500]
chang14_schedule_options = []
for lux_value in typical_indoor_lux_options:
    parameters = cp.deepcopy(chang14_parameters)
    parameters["typical_indoor_lux"] = lux_value
    chang14_schedule_options.append(parameters)

chang14_save_names = [f"chang14_light_indoor_lux_{lux_value}" for lux_value in typical_indoor_lux_options]

In [19]:
# Time
np.save(f"{save_path}/time.npy", time)

In [20]:
# Regular
schedule = LightSchedule.Regular(**regular_parameters)
light = schedule(time)
np.save(f"{save_path}/{regular_save_name}_light.npy", light)

In [21]:
# Chang14
def calculate_light_schedules(idx):
    print(f"Processing idx: {idx}")
    parameters = chang14_schedule_options[idx]
    save_name = chang14_save_names[idx]
    schedule_ebook_first = LightSchedule.Chang14(**parameters,
                                                 first_reading_condition="eReader")
    light_ebook_first = schedule_ebook_first(time)
    np.save(f"{save_path}/{save_name}_ebook_first_light.npy", light_ebook_first,)
    schedule_ebook_second = LightSchedule.Chang14(**parameters, 
                                                  first_reading_condition="Book")
    light_ebook_second = schedule_ebook_second(time)
    np.save(f"{save_path}/{save_name}_ebook_second_light.npy", light_ebook_second)

pool_values = range(len(typical_indoor_lux_options))
with ThreadPoolExecutor() as executor:
    results = list(executor.map(calculate_light_schedules, pool_values))

Processing idx: 0
Processing idx: 1
Processing idx: 2
Processing idx: 3
Processing idx: 4
Processing idx: 5
